# Importação dos recursos:

In [92]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [37]:
df = pd.read_csv('/kaggle/input/cars-sec-6/autos.csv', encoding='ISO-8859-1')
df

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371523,2016-03-14 17:48:27,Suche_t4___vito_ab_6_sitze,privat,Angebot,2200,test,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,2016-03-14 00:00:00,0,39576,2016-04-06 00:46:52
371524,2016-03-05 19:56:21,Smart_smart_leistungssteigerung_100ps,privat,Angebot,1199,test,cabrio,2000,automatik,101,fortwo,125000,3,benzin,smart,nein,2016-03-05 00:00:00,0,26135,2016-03-11 18:17:12
371525,2016-03-19 18:57:12,Volkswagen_Multivan_T4_TDI_7DC_UY2,privat,Angebot,9200,test,bus,1996,manuell,102,transporter,150000,3,diesel,volkswagen,nein,2016-03-19 00:00:00,0,87439,2016-04-07 07:15:26
371526,2016-03-20 19:41:08,VW_Golf_Kombi_1_9l_TDI,privat,Angebot,3400,test,kombi,2002,manuell,100,golf,150000,6,diesel,volkswagen,NaN,2016-03-20 00:00:00,0,40764,2016-03-24 12:45:21


# Exclusão de atributos que provavelmente não influenciam no preço:

In [38]:
df = df.drop('dateCrawled', axis=1) #data que os dados do carro foram extraídos do site de vendas
df = df.drop('nrOfPictures', axis=1) #número de fotos do carro no anúncio
df = df.drop('dateCreated', axis=1) #data de criação dos registro do carro na base de dados
df = df.drop('postalCode', axis=1) #código postal do carro
df = df.drop('lastSeen', axis=1) #data da última visualização do anúncio por algum usuário

# Análise e exclusão de outros atributos:

In [39]:
df['name'].value_counts()

name
Ford_Fiesta                                              657
BMW_318i                                                 627
Opel_Corsa                                               622
Volkswagen_Golf_1.4                                      603
BMW_316i                                                 523
                                                        ... 
Audi_A4_Avant_Klima_Gruene_Plakette_TÜV_&AU_NEU_XENON      1
Renault_clio_in_gold_450VB_!!                              1
Fiat_Doblo_1.6_Multijet                                    1
Renault_Laguna_1                                           1
BMW_M135i_vollausgestattet_NP_52.720____Euro               1
Name: count, Length: 233531, dtype: int64

Dos 371528 registros, 233531 possuem um valor único para a coluna "name". Ou seja, ~63% dos registros têm valor único para esse atributo.<br>
Essa grande variação dos valores torna praticamente impossível que um modelo de ML encontre um padrão nessa dimensão, tornando-a dispensável para um estimador.

In [40]:
df = df.drop('name', axis=1)

In [41]:
df['seller'].value_counts()

seller
privat        371525
gewerblich         3
Name: count, dtype: int64

Aqui, temos o problema inverso da coluna "name". Praticamente todos os registros têm o mesmo valor para o atributo "seller".<br>
Isso faz com que seja praticamente impossível determinar o impacto dessa variável no preço, tornando essa coluna dispensável nessa base de dados.

In [42]:
df = df.drop('seller', axis=1)

In [43]:
df['offerType'].value_counts()

offerType
Angebot    371516
Gesuch         12
Name: count, dtype: int64

Para a coluna "offerType", tem-se o mesmo problema da coluna "seller".

In [44]:
df = df.drop('offerType', axis=1)

In [45]:
df['abtest'].value_counts()

abtest
test       192585
control    178943
Name: count, dtype: int64

Base parcial:

In [46]:
df

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN
1,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja
2,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN
3,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein
4,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein
...,...,...,...,...,...,...,...,...,...,...,...,...
371523,2200,test,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN
371524,1199,test,cabrio,2000,automatik,101,fortwo,125000,3,benzin,smart,nein
371525,9200,test,bus,1996,manuell,102,transporter,150000,3,diesel,volkswagen,nein
371526,3400,test,kombi,2002,manuell,100,golf,150000,6,diesel,volkswagen,NaN


# Lidando com valores inconsistentes

In [47]:
df.loc[df['price'] <= 10].value_counts()

price  abtest   vehicleType  yearOfRegistration  gearbox  powerPS  model    kilometer  monthOfRegistration  fuelType  brand       notRepairedDamage
0      test     limousine    1997                manuell  75       golf     150000     0                    benzin    volkswagen  nein                 28
       control  limousine    1997                manuell  75       golf     150000     0                    benzin    volkswagen  nein                 28
       test     bus          2005                manuell  101      vivaro   80000      8                    diesel    opel        nein                 13
                kleinwagen   2012                manuell  90       rio      90000      7                    diesel    kia         nein                 10
       control  limousine    1998                manuell  60       golf     150000     0                    benzin    volkswagen  nein                  9
                                                                                  

Ou seja, há mais de 3400 registros nessa base de carros custando menos do que 10 Euros, o que não faz sentido. Possivelmente são valores extraídos de forma incorreta, ou registrados de forma errada pelo vendedor.<br>
De qualquer forma, essas valores não são interessantes para treinar um modelo que prevê o preço dos carros. 

In [48]:
df = df.loc[df['price'] > 10]

In [49]:
df.loc[df['price'] >= 350000].value_counts()

price     abtest   vehicleType  yearOfRegistration  gearbox    powerPS  model     kilometer  monthOfRegistration  fuelType  brand          notRepairedDamage
1250000   test     coupe        2016                manuell    500      911       5000       3                    benzin    porsche        nein                 2
350000    control  coupe        2016                manuell    500      911       5000       3                    benzin    porsche        nein                 1
9999999   control  coupe        1967                manuell    215      911       50000      10                   benzin    porsche        nein                 1
999990    test     limousine    1985                manuell    111      jetta     150000     12                   benzin    volkswagen     ja                   1
999999    test     kombi        2009                manuell    101      focus     125000     4                    benzin    ford           nein                 1
                   suv          1

Observa-se também registros de carros com valores muito altos. Alguns valores parecem frutos de erros, como "99999999".<br>
Também não são interessantes para o treinamento do modelo.

In [50]:
df = df.loc[df['price'] < 350000]

In [51]:
df

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN
1,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja
2,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN
3,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein
4,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein
...,...,...,...,...,...,...,...,...,...,...,...,...
371523,2200,test,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN
371524,1199,test,cabrio,2000,automatik,101,fortwo,125000,3,benzin,smart,nein
371525,9200,test,bus,1996,manuell,102,transporter,150000,3,diesel,volkswagen,nein
371526,3400,test,kombi,2002,manuell,100,golf,150000,6,diesel,volkswagen,NaN


# Lidando com valores faltantes

In [52]:
df.loc[pd.isnull(df['vehicleType'])]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN
16,300,test,NaN,2016,NaN,60,polo,150000,0,benzin,volkswagen,NaN
22,2900,test,NaN,2018,manuell,90,meriva,150000,5,benzin,opel,nein
26,5555,control,NaN,2017,manuell,125,c4,125000,4,NaN,citroen,nein
31,899,control,NaN,2016,manuell,60,clio,150000,6,benzin,renault,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
371495,180,control,NaN,1995,NaN,0,NaN,125000,3,benzin,opel,NaN
371504,2600,control,NaN,2005,automatik,0,c_klasse,150000,9,NaN,mercedes_benz,NaN
371509,1900,test,NaN,2000,manuell,110,NaN,150000,7,NaN,volkswagen,nein
371519,5250,control,NaN,2016,automatik,150,159,150000,12,NaN,alfa_romeo,nein


In [59]:
df['vehicleType'].mode()

0    limousine
Name: vehicleType, dtype: object

In [60]:
df.loc[pd.isnull(df['gearbox'])]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
15,450,test,kleinwagen,1910,NaN,0,ka,5000,0,benzin,ford,NaN
16,300,test,NaN,2016,NaN,60,polo,150000,0,benzin,volkswagen,NaN
32,245,test,limousine,1994,NaN,0,golf,150000,2,benzin,volkswagen,nein
37,1500,test,NaN,2016,NaN,0,kangoo,150000,1,diesel,renault,nein
70,1200,test,coupe,2001,NaN,0,astra,150000,0,NaN,opel,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
371443,3300,control,kombi,2006,NaN,0,touran,150000,7,diesel,volkswagen,NaN
371460,3500,control,NaN,1995,NaN,0,polo,150000,0,NaN,volkswagen,NaN
371486,350,control,kleinwagen,1996,NaN,65,punto,150000,0,NaN,fiat,NaN
371495,180,control,NaN,1995,NaN,0,NaN,125000,3,benzin,opel,NaN


In [61]:
df['gearbox'].mode()

0    manuell
Name: gearbox, dtype: object

In [62]:
df.loc[pd.isnull(df['model'])]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
1,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja
83,350,control,kleinwagen,1997,manuell,54,NaN,150000,3,NaN,fiat,ja
139,1450,control,limousine,1992,manuell,136,NaN,150000,0,NaN,audi,nein
156,6799,control,kleinwagen,2009,NaN,60,NaN,20000,5,benzin,volkswagen,nein
165,500,control,kleinwagen,1999,manuell,0,NaN,150000,0,benzin,renault,nein
...,...,...,...,...,...,...,...,...,...,...,...,...
371399,560,control,kleinwagen,2001,automatik,170,NaN,90000,0,benzin,fiat,ja
371476,9400,control,kombi,2007,manuell,200,NaN,150000,4,diesel,sonstige_autos,ja
371495,180,control,NaN,1995,NaN,0,NaN,125000,3,benzin,opel,NaN
371509,1900,test,NaN,2000,manuell,110,NaN,150000,7,NaN,volkswagen,nein


In [63]:
df['model'].mode()

0    golf
Name: model, dtype: object

In [64]:
df.loc[pd.isnull(df['fuelType'])]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
9,999,test,kleinwagen,1998,manuell,101,golf,150000,0,NaN,volkswagen,NaN
13,2500,control,kombi,2004,manuell,131,passat,150000,2,NaN,volkswagen,nein
26,5555,control,NaN,2017,manuell,125,c4,125000,4,NaN,citroen,nein
36,1600,control,andere,1991,manuell,75,kadett,70000,0,NaN,opel,NaN
41,7500,control,limousine,2002,automatik,306,e_klasse,150000,4,NaN,mercedes_benz,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
371496,3850,test,cabrio,2006,manuell,108,2_reihe,125000,2,NaN,peugeot,nein
371504,2600,control,NaN,2005,automatik,0,c_klasse,150000,9,NaN,mercedes_benz,NaN
371509,1900,test,NaN,2000,manuell,110,NaN,150000,7,NaN,volkswagen,nein
371519,5250,control,NaN,2016,automatik,150,159,150000,12,NaN,alfa_romeo,nein


In [65]:
df['fuelType'].mode()

0    benzin
Name: fuelType, dtype: object

In [67]:
df.loc[pd.isnull(df['notRepairedDamage'])]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN
2,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN
8,14500,control,bus,2014,manuell,125,c_max,30000,8,benzin,ford,NaN
9,999,test,kleinwagen,1998,manuell,101,golf,150000,0,NaN,volkswagen,NaN
12,999,control,kombi,1995,manuell,115,passat,150000,11,benzin,volkswagen,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
371507,5999,test,kombi,2005,manuell,140,a4,150000,4,diesel,audi,NaN
371514,999,control,cabrio,2000,manuell,95,megane,150000,4,benzin,renault,NaN
371515,1690,test,kombi,2004,manuell,55,fabia,150000,4,benzin,skoda,NaN
371523,2200,test,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN


In [68]:
df['notRepairedDamage'].mode()

0    nein
Name: notRepairedDamage, dtype: object

Como os atributos analisados acima são categórico, substituiremos os valores faltantes pela moda.<br>
Isso será feito pela função fillna:

In [69]:
values = {
    'vehicleType': 'limousine',
    'gearbox': 'manuell',
    'model': 'golf',
    'fuelType': 'benzin',
    'notRepairedDamage': 'nein'
}

In [70]:
df = df.fillna(value = values)

In [71]:
df

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,480,test,limousine,1993,manuell,0,golf,150000,0,benzin,volkswagen,nein
1,18300,test,coupe,2011,manuell,190,golf,125000,5,diesel,audi,ja
2,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,nein
3,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein
4,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein
...,...,...,...,...,...,...,...,...,...,...,...,...
371523,2200,test,limousine,2005,manuell,0,golf,20000,1,benzin,sonstige_autos,nein
371524,1199,test,cabrio,2000,automatik,101,fortwo,125000,3,benzin,smart,nein
371525,9200,test,bus,1996,manuell,102,transporter,150000,3,diesel,volkswagen,nein
371526,3400,test,kombi,2002,manuell,100,golf,150000,6,diesel,volkswagen,nein


In [74]:
df.isnull().sum()

price                  0
abtest                 0
vehicleType            0
yearOfRegistration     0
gearbox                0
powerPS                0
model                  0
kilometer              0
monthOfRegistration    0
fuelType               0
brand                  0
notRepairedDamage      0
dtype: int64

Como indicado acima, não há mais valores nulos no DataFrame df.

# Criando os conjuntos previsores e de classe

In [86]:
X = df.iloc[:, 1:12].values #todos os registros das colunas 1 a 11
y = df.iloc[:, 0].values #todos os registros da coluna 0

In [87]:
X

array([['test', 'limousine', 1993, ..., 'benzin', 'volkswagen', 'nein'],
       ['test', 'coupe', 2011, ..., 'diesel', 'audi', 'ja'],
       ['test', 'suv', 2004, ..., 'diesel', 'jeep', 'nein'],
       ...,
       ['test', 'bus', 1996, ..., 'diesel', 'volkswagen', 'nein'],
       ['test', 'kombi', 2002, ..., 'diesel', 'volkswagen', 'nein'],
       ['control', 'limousine', 2013, ..., 'benzin', 'bmw', 'nein']],
      dtype=object)

In [83]:
df.columns

Index(['price', 'abtest', 'vehicleType', 'yearOfRegistration', 'gearbox',
       'powerPS', 'model', 'kilometer', 'monthOfRegistration', 'fuelType',
       'brand', 'notRepairedDamage'],
      dtype='object')

Várias dessas colunas são de valores categóricos não ordinais. Esses precisam ser convertidos para valores One Hot para que possam ser compatíveis com a estruturas de uma rede neural, já que uma simples codificação por inteiros faria com que alguns valores tivessem mais peso que outros.<br>
São eles: abtest (0), vehicleType (1), gearbox(3), model (5), fuelType (8), brand (9), notRepairedDamage (10).<br>
yearOfRegistration e monthOfRegistration são ordinais, por isso não serão convertidos para One Hot.

In [84]:
onehotencoder = ColumnTransformer(transformers = [("OneHot", OneHotEncoder(), [0, 1, 3, 5, 8, 9, 10])], remainder = 'passthrough')

In [88]:
X = onehotencoder.fit_transform(X).toarray() #aplica a transformação definida em onehotencoder em X e converte os resultados para array
X

array([[0.00e+00, 1.00e+00, 0.00e+00, ..., 0.00e+00, 1.50e+05, 0.00e+00],
       [0.00e+00, 1.00e+00, 0.00e+00, ..., 1.90e+02, 1.25e+05, 5.00e+00],
       [0.00e+00, 1.00e+00, 0.00e+00, ..., 1.63e+02, 1.25e+05, 8.00e+00],
       ...,
       [0.00e+00, 1.00e+00, 0.00e+00, ..., 1.02e+02, 1.50e+05, 3.00e+00],
       [0.00e+00, 1.00e+00, 0.00e+00, ..., 1.00e+02, 1.50e+05, 6.00e+00],
       [1.00e+00, 0.00e+00, 0.00e+00, ..., 3.20e+02, 5.00e+04, 8.00e+00]])

In [90]:
df.shape

(359291, 12)

In [91]:
X.shape

(359291, 316)

Antes, o conjunto de atributos previsores tinha 11 colunas (todas do conjunto total menos a de preço), agora tem 316.<br>
Isso se deve ao fato de que cada atributo passou a ser representado por um número de colunas igual ao número de valores únicos (classes) associados a ele, pois é assim que funciona a codificação One Hot.<br>
Ou seja, agora cada amostra (registro de carro) é representada por uma array numérico, em que alguns números representam um atributo e outros representam a validade ou não de uma classe de um atributo (se o elemento é ou não daquela classe).

# Criando e treinando a rede neural 

In [94]:
regressor = Sequential()
regressor.add(InputLayer(shape=(316,)))
regressor.add(Dense(units = 158, activation='relu'))
regressor.add(Dense(units = 158, activation='relu'))
regressor.add(Dense(units = 1, activation='linear'))

regressor.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mean_absolute_error'])

In [95]:
regressor.fit(X, y, batch_size=300, epochs=100)

Epoch 1/100
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 4216.1890 - mean_absolute_error: 4216.1890
Epoch 2/100
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 3435.9988 - mean_absolute_error: 3435.9988
Epoch 3/100
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 3136.5181 - mean_absolute_error: 3136.5181
Epoch 4/100
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 2883.0081 - mean_absolute_error: 2883.0083
Epoch 5/100
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 2804.3467 - mean_absolute_error: 2804.3467
Epoch 6/100
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 2744.2881 - mean_absolute_error: 2744.2881
Epoch 7/100
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 2712.6765 - mean_absolute_error: 2712.6765
Epoch 8/100
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 2676.9014 - mean_absolute_error: 2676.9014
Epoch 9/100
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 2639.1624 - mean_absolute_error: 2639.1624
Epoch 10/100
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 4s

In [96]:
predictions = regressor.predict(X)

11228/11228 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step


In [97]:
predictions

array([[ 1715.4434],
       [10420.064 ],
       [12934.596 ],
       ...,
       [ 5485.8823],
       [ 3402.078 ],
       [27403.395 ]], dtype=float32)

In [98]:
y

array([  480, 18300,  9800, ...,  9200,  3400, 28990])

In [99]:
predictions.mean()

5556.607

In [100]:
y.mean()

5916.833945186492